In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ======================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [2]:
import os
import glob

import numpy as np
import pandas as pd

import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

### Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = os.environ.get("NUM_ROWS", 100000)

In [5]:
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., int(NUM_ROWS)).astype(np.int32), 1, 50000)
# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, int(NUM_ROWS)), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['age_days'] = np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)
df['weekday_sin']= np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [6]:
df.head()

,session_id,item_id,category,age_days,weekday_sin,day
0,75789,8,2,0.358370,0.707811,1
1,86997,43,9,0.577899,0.840965,1
2,71211,3,1,0.198717,0.119433,4
3,73360,10,2,0.791937,0.146101,5
4,86785,18,4,0.623115,0.617554,3


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers. Note that the `Categorify` op encodes OOVs or nulls to `0` automatically. In our synthetic dataset we do not have any nulls. On the other hand `0` is also used for padding the sequences in input block, therefore, you can set `start_index=1` arg in the Categorify op if you want the encoded null or OOV values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

Here our goal is to create sequential features. To do so, we are grouping the features together at the session level in the following cell. In this synthetically generated example dataset, we do not have a timestamp column, but if we had one (that's the case for most real-world datasets), we would be sorting the interactions by the timestamp column as in this [example notebook](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/end-to-end-session-based/01-ETL-with-NVTabular.ipynb). Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [7]:
SESSIONS_MAX_LENGTH =20

# Categorify categorical features
categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify()

# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'age_days', 'weekday_sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "age_days": ["list"],
        'weekday_sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['category-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
)

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['age_days-list', 'weekday_sin-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['item_id-list', 'category-list', 'age_days-list', 'weekday_sin-list'] >>  nvt.ops.ValueCount()


workflow = nvt.Workflow(filtered_sessions['session_id', 'day-first', 'item_id-count'] + seq_feats_list)

dataset = nvt.Dataset(df, cpu=False)
# Generate statistics for the features
workflow.fit(dataset)
# Apply the preprocessing and return an NVTabular dataset
sessions_ds = workflow.transform(dataset)
# Convert the NVTabular dataset to a Dask cuDF dataframe (`to_ddf()`) and then to cuDF dataframe (`.compute()`)
sessions_gdf = sessions_ds.to_ddf().compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [8]:
sessions_gdf.head(3)

,session_id,day-first,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,1,7,16,"[3, 12, 9, 34, 119, 21, 22, 9, 32, 18, 38, 9, ...","[1, 3, 2, 7, 26, 5, 5, 2, 3, 5, 8, 2, 6, 7, 4, 2]","[0.85696095, 0.4300088, 0.033559408, 0.2394981...","[0.7585879, 0.27019486, 0.8184247, 0.553261, 0..."
1,2,5,15,"[12, 43, 16, 45, 44, 6, 7, 10, 36, 20, 49, 27,...","[3, 8, 4, 10, 10, 1, 2, 2, 9, 3, 10, 6, 10, 12...","[0.37208363, 0.99748486, 0.14058138, 0.5112176...","[0.5443008, 0.67776006, 0.39591455, 0.61404943..."
2,3,2,15,"[11, 36, 12, 13, 11, 38, 7, 37, 2, 19, 8, 7, 1...","[2, 9, 3, 2, 2, 8, 2, 9, 1, 5, 3, 2, 1, 3, 2]","[0.56365025, 0.34677047, 0.94600457, 0.9574235...","[0.88817126, 0.47712854, 0.37194037, 0.2259221..."


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [9]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(Tags.CATEGORICAL),int64,False,False,NaN,0.0,0.0,0.0,.//categories/unique.session_id.parquet,0.0,19881.0,session_id,19882.0,409.0,NaN,NaN
1,day-first,(),int64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-count,(Tags.CATEGORICAL),int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,488.0,item_id,489.0,51.0,NaN,NaN
3,item_id-list,"(Tags.ITEM, Tags.LIST, Tags.ID, Tags.CATEGORIC...",int64,True,True,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,488.0,item_id,489.0,51.0,2.0,16.0
4,category-list,"(Tags.CATEGORICAL, Tags.LIST)",int64,True,True,NaN,0.0,0.0,0.0,.//categories/unique.category.parquet,0.0,144.0,category,145.0,26.0,2.0,16.0
5,age_days-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,16.0
6,weekday_sin-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,16.0


The following will generate `schema.pbtxt` file in the provided folder.

In [10]:
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [11]:
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow_etl"))

## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [12]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))

In [13]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 31.76it/s]


## Checking the preprocessed outputs

In [14]:
TRAIN_PATHS = os.path.join(OUTPUT_DIR, "1", "train.parquet")

In [15]:
df = pd.read_parquet(TRAIN_PATHS)
df

,session_id,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,6,14,"[55, 22, 1, 20, 25, 18, 75, 3, 15, 31, 13, 20,...","[13, 5, 1, 3, 6, 5, 15, 1, 4, 7, 2, 3, 3, 2]","[0.97995365, 0.067363486, 0.66040653, 0.494267...","[0.3487224, 0.9060633, 0.25998157, 0.0769127, ..."
1,23,13,"[161, 70, 8, 20, 5, 12, 32, 33, 31, 32, 47, 50...","[36, 16, 3, 3, 3, 3, 3, 7, 7, 3, 10, 11, 8]","[0.10111435, 0.47324365, 0.2752711, 0.17544255...","[0.8532496, 0.83872354, 0.993718, 0.72692555, ..."
2,32,13,"[3, 16, 8, 18, 4, 2, 49, 35, 9, 8, 79, 67, 2]","[1, 4, 3, 5, 1, 1, 10, 9, 2, 3, 17, 14, 1]","[0.5600003, 0.11820206, 0.6599864, 0.005933116...","[0.5309871, 0.71518654, 0.5602452, 0.37133983,..."
4,36,13,"[5, 1, 2, 2, 20, 28, 282, 73, 25, 68, 40, 146, 2]","[3, 1, 1, 1, 3, 7, 54, 16, 6, 16, 8, 30, 1]","[0.8373534, 0.35135373, 0.54747295, 0.91502804...","[0.37596887, 0.057511043, 0.22147927, 0.270423..."
5,40,13,"[2, 3, 38, 2, 4, 12, 32, 13, 64, 39, 10, 32, 9]","[1, 1, 8, 1, 1, 3, 3, 2, 12, 9, 2, 3, 2]","[0.6041386, 0.92545164, 0.9951186, 0.9442425, ...","[0.65229225, 0.11658515, 0.17234567, 0.1389862..."
...,...,...,...,...,...,...
2130,19171,2,"[47, 28]","[10, 7]","[0.023725191, 0.87022305]","[0.6921467, 0.50588226]"
2131,19174,2,"[1, 23]","[1, 5]","[0.7369641, 0.033553768]","[0.56077486, 0.8527744]"
2132,19180,2,"[8, 49]","[3, 10]","[0.48836, 0.5168656]","[0.9988041, 0.026354417]"
2133,19189,2,"[7, 4]","[2, 1]","[0.53906214, 0.6020796]","[0.6170275, 0.080708236]"


In [16]:
import gc
del df
gc.collect()

443

You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model. Please shut down this kernel to free the GPU memory before you start the next one.